In [ ]:
#InceptonResnetV2 code:
import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
 
# Set the number of classes for rice plant diseases
num_classes = 9
 
# Load the pre-trained InceptionResNetV2 model without the top (fully connected) layers
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
 
# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
 
# Freeze the base layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False
 
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
 
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
)
 
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    'D://DEEP LEARNING DATASET/training',
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)
 
validation_generator = test_datagen.flow_from_directory(
    'D://DEEP LEARNING DATASET/testing',
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)
 
# Fine-tuning: Unfreeze some layers for further training
for layer in model.layers[-50:]:
    layer.trainable = True
 
# Re-compile the model after fine-tuning
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
 
# Define a ModelCheckpoint callback to save the best weights during training
checkpoint = ModelCheckpoint(
    'best_weights.h5',
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    mode='max'
)
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=25,  # You can adjust the number of fine-tuning epochs as needed
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size,
    callbacks=[checkpoint]
)

import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid()
plt.show()
